## Prepare dataloader

In [ ]:
from utils.prepare_dataset import prepare_data

prepare_data()

### Training

In [ ]:
import time
from training import train

In [ ]:
print("### Starting training... ###")
t0 = time.time()
train(stud_id=1)
t1 = time.time()
print("training/validation time: {0:.2f}s".format(t1 - t0))
print("### DataLoader ready ###")